In [10]:
### string.fromat

import yaml
import import_ipynb
import Read_pdf as rp

def display_template(data):
    topic = data.keys()
    print('--------------模板列表-----------------')
    for item in topic:
        print(item + ':')
        for sub_item in data[item]:
            print(' -' + sub_item)

def get_variable(text):
    my_pos = []
    my_pos2 = []
    pos = 0
    pos2 = 0
    while True:
        pos = text.find('{', pos)
        pos2 = text.find('}', pos2)
        if pos2 == -1:
            break
        my_pos.append(pos)
        my_pos2.append(pos2)
        pos += 1
        pos2 +=1

    keys = []
    for start,end in zip(my_pos, my_pos2):
        keys.append(text[start+1:end])
    d = dict.fromkeys(keys)
    del my_pos
    del my_pos2
    del keys
    return d


def bid_value(bid, dic):
    awd_sn = 0 #表示決包廠商序號

    if bid.currency == '人民幣':
        prefixCur = 'RMB'
        suffixCur = ''
    else:
        prefixCur =''
        suffixCur = '元'

    def name_format(var):
        return '「' + str(var) +'」'

    def price_format(var):
        return prefixCur + str(var) + suffixCur

    for key in dic:
        if key == '次別':
            dic[key] = bid.sn
        elif key == '決包廠商':
            dic[key] = name_format(bid.award_vnd)
        elif key == '追加減':
            dic[key] = bid.change
        elif key == '預算金額':
            dic[key] = price_format(bid.budget)
        elif key == '報價金額':
            dic[key] = price_format(bid.vnd[awd_sn]['quo'])
        elif key == '議價金額':
            dic[key] = price_format(bid.vnd[awd_sn]['finalPrice'])
        elif key == '差異':
            dic[key] = bid.dif_text(awd_sn)
        else:
            dic[key] = input('請輸入「'+key +'」的值:')
    return dic

if __name__ == '__main__':
    file_yaml = 'd:/template.yaml'
    with open(file_yaml, 'r', encoding = 'utf8') as stream:
        data = yaml.load(stream, Loader = yaml.FullLoader)
    
    print('--------------模板可用樣式列表-------------')
    display_template(data)        

    print('--------------模板樣式-------------')
    text = data['預約子案']['有議價']
    print(text)
    
    pdf_file = 'd:/S1.pdf'
    bidcase = rp.open(pdf_file)

    dict_tags = get_variable(text)
    dict_tags = bid_value(bidcase, dict_tags)

    print('----------------套版後-------------------')
    print(text.format(**dict_tags))

--------------模板可用樣式列表-------------
--------------模板列表-----------------
變更案:
 -有議價
 -無議價
 -簡易版
補辦案:
 -有議價
 -無議價
 -簡易版
預約子案:
 -有議價
 -無議價
 -簡易版
一般案:
 -有議價(低預算)
 -無議價(低預算)
 -有議價(超預算)
--------------模板樣式-------------
1.本案為{公司別}{廠區}「{預約工程}」之子案，遴選合約商{決包廠商}施作，預算編列{預算金額}。
2.經洽議{決包廠商}後由{報價金額}降至{議價金額}，{差異}。
3.擬以{報價金額}交{決包廠商}承攬(同母案開立{x稅率}%增值稅發票)。

請輸入「公司別」的值:XX
請輸入「廠區」的值:EE
請輸入「預約工程」的值:RRR
請輸入「x稅率」的值:3
----------------套版後-------------------
1.本案為XXEE「RRR」之子案，遴選合約商「厦门华雅建设」施作，預算編列RMB 29,838。
2.經洽議「厦门华雅建设」後由RMB 40,222降至RMB 29,838，同預算價格。
3.擬以RMB 40,222交「厦门华雅建设」承攬(同母案開立3%增值稅發票)。

